In [1]:
import sys, os
import re
import time
import gzip
import shutil, urllib, subprocess

from Bio import Entrez
import pandas as pd
import numpy as np
from os.path import join as pjoin


In [2]:
def get_summary(id, database):
    print(id)
    from Bio import Entrez
    Entrez.email = 'thomas_jordan@hms.harvard.edu'
    """Get esummary for an entrez id"""
    esummary_handle = Entrez.esummary(db = database, id = id, report = 'full', retmode = 'xml')
    esummary_record = Entrez.read(esummary_handle, validate = False)
    return esummary_record

In [3]:
def what_format(accession):
    if( re.search('[A-Z]{5}[0-9]{7}', accession) ):
        return 'mga'
    elif( re.search('[A-Z]{4}[0-9]{2}[0-9]{6,}', accession) or re.search('[A-Z]{6}[0-9]{2}[0-9]{7,}', accession) ):
        return 'wgs'
    elif( re.search('[A-Z]{1}[0-9]{5}', accession) or re.search('[A-Z]{2}[0-9]{6}', accession) or re.search('[A-Z]{2}[0-9]{8}', accession) ):
        return 'nucleotide'
    else:
        return 'other'

In [4]:
def search_metadata(metadata):
    total_length_string = '<Stat category="total_length" sequence_tag="all">[0-9]{3,}</Stat>'
    sequence_length = re.findall(total_length_string, metadata)[0].replace('<Stat category="total_length" sequence_tag="all">', '').replace('</Stat>', '')
    return int(sequence_length)

In [5]:
def bad_keywords(string):
    # returns true if bad keyword metagenome is found
    return bool( re.search('metagenome', string) or re.search('Metagenome', string) )

In [27]:
def get_genome(accession, genome_file):
    Entrez.email = 'thomas_jordan@hms.harvard.edu'

    print('New accession number: ' + accession)
    accession_type = what_format(accession)
    print('Accession number ' + str(accession) + ' is of the type ' + accession_type)

    link_handle = Entrez.elink(dbfrom = 'nucleotide', db = 'assembly', id = accession)
    linked_records = Entrez.read(link_handle)
    time.sleep(0.34)
    initial_summary = Entrez.read(Entrez.esummary(db = 'nucleotide', id = accession, retmode = 'xml'))[0]
    
    #print('Linked records: ' + str(linked_records))
    if(len(linked_records) != 0):    
        id = linked_records[0].get('LinkSetDb', False)
        if(len(id) == 0): # if there are links but the linksetdb dict is empty
            id = False
        if(id != False):
            id = id[0].get('Link', False)
            if(len(id) == 0): # if there are links but the link sub dict is empty
                id = False
            if(id != False):
                id = id[0].get('Id', False) # id of the assembly
    elif(len(linked_records) == 0):
        id = False
        
    if( id == False or bad_keywords(initial_summary['Title']) == True):
        # fallback method takes input accession and grabs the sequence associated to it from nucleotide
        if(id == False and bad_keywords(initial_summary['Title']) == True):
            print('Did not find unique accession number in assembly linked to ' + str(accession) + '\nSummary contains illegal keywords, using fallback method')
        elif(id == False):
            print('Did not find unique accession number in assembly linked to ' + str(accession) + '\nUsing fallback option')
        elif(bad_keywords(initial_summary['Title']) == True):
            print('Summary for ' + str(accession) + ' contains illegal keywords\nUsing fallback option')
        
        handle = Entrez.efetch(db = 'nucleotide', id = accession, rettype = 'fasta', retmode = 'text')
        sequence_length = int(re.sub('[^0-9]', '', str(initial_summary.get('Length'))))
        
        with open(genome_file, 'w') as fp:
            fp.write(handle.read())
            handle.close()
        
        print('Downloaded sequence fasta for ' + accession + ' from nucleotide')
        time.sleep(0.34)
        return 'nucleotide', sequence_length
    
    print('Found unique ID number [UID]: ' + str(id) + ' from ID list: ' + str(linked_records[0]['LinkSetDb'][0]))
        
    summary = pd.DataFrame(get_summary(id, 'assembly'))
    if(len(summary) == 0):
        # if entrez cannot fetch the summary for the linked entry
        print('Cannot find summary for linked assembly' + str(accession) + '\nUsing fallback method')
        handle = Entrez.efetch(db = 'nucleotide', id = accession, rettype = 'fasta', retmode = 'text')
        sequence_length = int(re.sub('[^0-9]', '', str(initial_summary.get('Length'))))
        
        with open(genome_file, 'w') as fp:
            fp.write(handle.read())
            handle.close()
        
        print('Downloaded sequence fasta for ' + accession + ' from nucleotide')
        time.sleep(0.34)
        return 'nucleotide', sequence_length
    
    #print(summary)    
    summary = summary['DocumentSummarySet']['DocumentSummary'][0]
    sequence_length = search_metadata(summary['Meta'])
    # checks if the assembly is excluded from RefSeq
    exclusion = bool(summary.get('ExclFromRefSeq')) # False if not excluded
    
    if(exclusion == False): # if this is none and the assembly is not excluded from RefSeq
        print('Assembly is contained in RefSeq')
        #get ftp link for RefSeq sequence
        url = summary['FtpPath_RefSeq']
        label = os.path.basename(url)
        #get the fasta link - change this to get other formats
        link = os.path.join(url, label + '_genomic.fna.gz')   
        print('genome link: ' + link)
        urllib.request.urlretrieve(link, f'genome.fna.gz')
        with gzip.open('genome.fna.gz', 'rb') as f_in:
            with open(genome_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove('genome.fna.gz')
        print('Downloaded genome assembly fasta for ' + accession + ' from RefSeq')
        time.sleep(0.34)
        return 'RefSeq', sequence_length
    
    elif(exclusion == True):
        print('Assembly is not contained in RefSeq\nReasons for exclusion: ' + str(summary['ExclFromRefSeq']))
        genbank_ftp = bool(summary.get('FtpPath_GenBank'))   
        
        if(genbank_ftp == True and summary['FtpPath_GenBank'] != ''):
            print('Assembly has link to GenBank')
            #get ftp link for GenBank sequence
            url = summary['FtpPath_GenBank']
            label = os.path.basename(url)
            #get the fasta link - change this to get other formats
            link = os.path.join(url, label + '_genomic.fna.gz')   
            print('genome link: ' + link)
            urllib.request.urlretrieve(link, f'genome.fna.gz')
            with gzip.open('genome.fna.gz', 'rb') as f_in:
                with open(genome_file, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            os.remove('genome.fna.gz')
            print('Downloaded genome assembly fasta for ' + str(accession) + ' from GenBank')
            time.sleep(0.34)
            return 'GenBank', sequence_length
                    
        elif(genbank_ftp == False):
            # fallback method takes inputaccession and grabs the sequence associated to it
            print('Assembly has no link to RefSeq or GenBank, using fallback method')
            handle = Entrez.efetch(db = 'nucleotide', id = accession, rettype = 'fasta', retmode = 'text')
            summary = pd.DataFrame(get_summary(id, 'nucleotide'))['DocumentSummarySet']['DocumentSummary'][0]
            sequence_length = search_metadata(summary['total_length'])
            
            with open(genome_file, 'w') as fp:
                fp.write(handle.read())
                handle.close()
                
            print('Downloaded sequence fasta for ' + str(accession) + ' from nucleotide')
            time.sleep(0.34)
            return 'nucleotide', sequence_length
                          

In [28]:
def call_barrnap(accession, genome_file):   
    ## call barrnap
    print('Calling barrnap for %s...' % accession)
    arguments = 'barrnap -k bac --outseq output.fa <' + genome_file
    barrnap_call = subprocess.Popen(arguments, shell = True)
    barrnap_call.wait()
    print('Barrnap done')
    headers = []
    sequences = []
    with open('output.fa', 'r') as fp:
        for line in fp:
            if('>16S_' in str(line)):
                headers.append(line.strip().replace('>', '').replace('\n', '').replace('>16S_rRNA::', ''))
                line = fp.readline().replace('\n', '')
                sequences.append(line)
    if(len(headers) != 0):
        print('Found ' + str(len(headers)) + ' hits using barrnap\n\n')
        sys.stdout.flush()
    elif(len(headers) == 0):
        print('Found no hits using barrnap\n\n')
        sys.stdout.flush()
    return headers, sequences

In [33]:
def run_functions(accession):
    try:
        source, sequence_length = get_genome(accession, 'genome.fa')
        headers, sequences = call_barrnap(accession, 'genome.fa')
        return headers, sequences, source, sequence_length, None
    except urllib.error.URLError as e:
        return None, None, None, None, e
    except ValueError as e:
        return None, None, None, None, e
    except ConnectionResetError as e:
        return None, None, None, None, e

In [34]:

#test cases
# df = pd.read_csv('unique_accessions.csv')
# dfd = df.sample(100)

# dfd[['16s_header', '16s_sequence', 'source', 'barrnap_input_sequence_length', 'errors']] = dfd.apply(lambda x: run_functions(x.accession_name), axis = 'columns', result_type = 'expand')

# display(dfd)

# run_functions('GL870821')
# run_functions('AAEK01000042')
# run_functions('FWXI01000015')
# run_functions('ADGO01007773')
# run_functions('BAUV01000014')
# run_functions('FR893325')
# run_functions('AFGF01000211')
# run_functions('ABEF01053366')
# run_functions('CP028421')

New accession number: JXCY01000004
Accession number JXCY01000004 is of the type wgs
Found unique ID number [UID]: 479801 from ID list: {'Link': [{'Id': '479801'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly'}
479801
Assembly is contained in RefSeq
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/001/281/205/GCF_001281205.1_ASM128120v1/GCF_001281205.1_ASM128120v1_genomic.fna.gz
Downloaded genome assembly fasta for JXCY01000004 from RefSeq
Calling barrnap for JXCY01000004...
Barrnap done
Found 1 hits using barrnap


New accession number: HF995001
Accession number HF995001 is of the type nucleotide
Found unique ID number [UID]: 431251 from ID list: {'Link': [{'Id': '431251'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly'}
431251
Assembly is not contained in RefSeq
Reasons for exclusion: ['derived from environmental source']
Assembly has link to GenBank
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/432/795/GCA_000432795.1_MGS235/GCA_000432795.1_MGS235_geno

Found unique ID number [UID]: 291458 from ID list: {'Link': [{'Id': '291458'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly'}
291458
Assembly is contained in RefSeq
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/218/425/GCF_000218425.1_Lach_bact_5_1_57FAA/GCF_000218425.1_Lach_bact_5_1_57FAA_genomic.fna.gz
Downloaded genome assembly fasta for GL945248 from RefSeq
Calling barrnap for GL945248...
Barrnap done
Found 2 hits using barrnap


New accession number: AEUN01000192
Accession number AEUN01000192 is of the type wgs
Found unique ID number [UID]: 318848 from ID list: {'Link': [{'Id': '318848'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly'}
318848
Assembly is contained in RefSeq
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/235/645/GCF_000235645.1_ASM23564v1/GCF_000235645.1_ASM23564v1_genomic.fna.gz
Downloaded genome assembly fasta for AEUN01000192 from RefSeq
Calling barrnap for AEUN01000192...
Barrnap done
Found 1 hits using barrnap


New access

Assembly is not contained in RefSeq
Reasons for exclusion: ['derived from environmental source']
Assembly has link to GenBank
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/431/815/GCA_000431815.1_MGS318/GCA_000431815.1_MGS318_genomic.fna.gz
Downloaded genome assembly fasta for FR889242 from GenBank
Calling barrnap for FR889242...
Barrnap done
Found no hits using barrnap


New accession number: AKKV01000034
Accession number AKKV01000034 is of the type wgs
Found unique ID number [UID]: 395288 from ID list: {'Link': [{'Id': '395288'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly'}
395288
Assembly is contained in RefSeq
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/269/865/GCF_000269865.1_ASM26986v1/GCF_000269865.1_ASM26986v1_genomic.fna.gz
Downloaded genome assembly fasta for AKKV01000034 from RefSeq
Calling barrnap for AKKV01000034...
Barrnap done
Found 1 hits using barrnap


New accession number: MJIH01000001
Accession number MJIH01000001 is of the type

New accession number: GL635750
Accession number GL635750 is of the type nucleotide
Found unique ID number [UID]: 241638 from ID list: {'Link': [{'Id': '241638'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly'}
241638
Assembly is contained in RefSeq
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/186/145/GCF_000186145.1_Bacillus_sp_2_A_57_CT2_V1/GCF_000186145.1_Bacillus_sp_2_A_57_CT2_V1_genomic.fna.gz
Downloaded genome assembly fasta for GL635750 from RefSeq
Calling barrnap for GL635750...
Barrnap done
Found 2 hits using barrnap


New accession number: ADXE01001406
Accession number ADXE01001406 is of the type wgs
Found unique ID number [UID]: 460108 from ID list: {'Link': [{'Id': '460108'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly_wgscontig'}
460108
Assembly is not contained in RefSeq
Reasons for exclusion: ['fragmented assembly']
Assembly has link to GenBank
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/183/845/GCA_000183845.1_ASM18384v1/GCA_000

Assembly is contained in RefSeq
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/808/015/GCF_000808015.1_pear-SPAdes-1.0/GCF_000808015.1_pear-SPAdes-1.0_genomic.fna.gz
Downloaded genome assembly fasta for JWHR01000115 from RefSeq
Calling barrnap for JWHR01000115...
Barrnap done
Found 1 hits using barrnap


New accession number: HG917868
Accession number HG917868 is of the type nucleotide
Found unique ID number [UID]: 121721 from ID list: {'Link': [{'Id': '121721'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly'}
121721
Assembly is contained in RefSeq
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/577/895/GCF_000577895.1_M2_40/GCF_000577895.1_M2_40_genomic.fna.gz
Downloaded genome assembly fasta for HG917868 from RefSeq
Calling barrnap for HG917868...
Barrnap done
Found 8 hits using barrnap


New accession number: AATD01005631
Accession number AATD01005631 is of the type wgs
Did not find unique accession number in assembly linked to AATD01005631
Summary cont

Downloaded genome assembly fasta for ACTX01000006 from RefSeq
Calling barrnap for ACTX01000006...
Barrnap done
Found 1 hits using barrnap


New accession number: KN125580
Accession number KN125580 is of the type nucleotide
Found unique ID number [UID]: 207041 from ID list: {'Link': [{'Id': '207041'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly'}
207041
Assembly is contained in RefSeq
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/746/875/GCF_000746875.1_BMM_2/GCF_000746875.1_BMM_2_genomic.fna.gz
Downloaded genome assembly fasta for KN125580 from RefSeq
Calling barrnap for KN125580...
Barrnap done
Found 5 hits using barrnap


New accession number: CYSP01000018
Accession number CYSP01000018 is of the type wgs
Found unique ID number [UID]: 488761 from ID list: {'Link': [{'Id': '488761'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly'}
488761
Assembly is contained in RefSeq
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/001/298/735/GCF_001298735.1_2_2/GCF_

Downloaded genome assembly fasta for JPVT01000244 from RefSeq
Calling barrnap for JPVT01000244...
Barrnap done
Found 1 hits using barrnap


New accession number: AAOX01000016
Accession number AAOX01000016 is of the type wgs
Found unique ID number [UID]: 177738 from ID list: {'Link': [{'Id': '177738'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly_wgscontig'}
177738
Assembly is contained in RefSeq
genome link: ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/153/365/GCF_000153365.1_ASM15336v1/GCF_000153365.1_ASM15336v1_genomic.fna.gz
Downloaded genome assembly fasta for AAOX01000016 from RefSeq
Calling barrnap for AAOX01000016...
Barrnap done
Found 9 hits using barrnap


New accession number: HF991874
Accession number HF991874 is of the type nucleotide
Found unique ID number [UID]: 41981 from ID list: {'Link': [{'Id': '41981'}], 'DbTo': 'assembly', 'LinkName': 'nuccore_assembly'}
41981
Assembly is not contained in RefSeq
Reasons for exclusion: ['derived from environmental source']
As

,accession_name,16s_header,16s_sequence,source,barrnap_input_sequence_length,errors
2717,JXCY01000004,[16S_rRNA::NZ_JXCY01000001.1:75338-76907(+)],[ATGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCCT...,RefSeq,1552941.0,None
1884,HF995001,[16S_rRNA::HF994942.1:0-589(+)],[ATGTGGTTTAATTCGAAGCAACGCGAAGAACCTTACCAGGTCTTG...,GenBank,2353225.0,None
8797,AAQK01008778,[],[],nucleotide,2134.0,None
5642,LDYG01000059,[],[],RefSeq,3407443.0,None
6019,JSCE01000250,"[16S_rRNA::NZ_JSCE01000260.1:5-755(-), 16S_rRN...",[TTGGTGGGGTAACGGCCCACCAAGGCGACGATCAGTAGCCGGTCT...,RefSeq,2828534.0,None
...,...,...,...,...,...,...
3672,AAJM01000527,"[16S_rRNA::AAJM01000140.1:1718-3268(+), 16S_rR...",[ATTGGAGAGTTTGATCCTGGCTCAGGATGAACGCTGGCGGCGTGC...,GenBank,5880839.0,None
2022,FR891331,[],[],GenBank,2111088.0,None
3527,JH601091,"[16S_rRNA::NZ_JH601091.1:84-1637(-), 16S_rRNA:...",[TTTGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGC...,RefSeq,2562512.0,None
5691,LHUR01000042,[16S_rRNA::NZ_LHUR01000014.1:1-1503(+)],[AGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCCTAAC...,RefSeq,3650535.0,None


In [ ]:
for index, chunk in enumerate(pd.read_csv('unique_accessions.csv', error_bad_lines=False, chunksize = 1000)):
    chunk[['16s_header', '16s_sequence', 'source', 'barrnap_input_sequence_length', 'errors']] = chunk.apply(lambda x: get_16s(x['accession_name']), axis = 'columns', result_type = 'expand')
    name = '16s_barrnap_' + str(index) + '.csv'
    print('chunk ', index, ' done')
    chunk.to_csv(name, header = True, index = False)